# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 40 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.11704


extracting tarball to tmp_2208.11704...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.11709


extracting tarball to tmp_2208.11709...

...

extracting tarball to tmp_2208.11871...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.12089


extracting tarball to tmp_2208.12089...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3398: LatexWarning: Found documentclass in tmp_2208.12089/NEOWISEpaper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'D+' from 'tmp_2208.12089/D+.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:292: LatexWarning: Latex injecting: 'optdepth' from 'tmp_2208.12089/optdepth.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/late

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11704-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11704) | **Weighing the Local Interstellar Medium using Gamma Rays and Dust**  |
|| Axel Widmark, Michael Korsmeier, Tim Linden |
|*Appeared on*| *2022-08-26*|
|*Comments*| *5 pages, 3 figures; appendix adds 16 pages, 9 figures; to be submitted to PRL, comments are welcome*|
|**Abstract**| Cold gas forms a significant mass fraction of the Milky Way disk, but is its most uncertain baryonic component. The density and distribution of cold gas is of critical importance for Milky Way dynamics, as well as models of stellar and galactic evolution. Previous studies have used correlations between gas and dust to obtain high-resolution measurements of cold gas, but with large normalization uncertainties. We present a novel approach that uses Fermi-LAT $\gamma$-ray data to measure the total gas density, achieving a similar precision as previous works, but with independent systematic uncertainties. Notably, our results have sufficient precision to distinguish between the tension in current world-leading experiments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11709-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11709) | **Statistical Mechanics of Collisionless Orbits. V. The approach to  equilibrium for idealized self-gravitating systems**  |
|| <mark>Liliya L.R. Williams</mark>, Jens Hjorth |
|*Appeared on*| *2022-08-26*|
|*Comments*| *12 pages, 6 figures, 1 appendix, accepted to ApJ*|
|**Abstract**| Self-gravitating Newtonian systems consisting of a very large number of particles have generally defied attempts to describe them using statistical mechanics. This is paradoxical since many astronomical systems, or simulations thereof, appear to have universal, equilibrium structures for which no physical basis exist. A decade ago we showed that extremizing the number of microstates with a given energy per unit mass, under the constraints of conserved total energy and mass, leads to the maximum entropy state, $n(E) \propto \exp (-\beta(E-\Phi_0))-1$, known as DARKexp. This differential energy distribution, and the resulting density structures, closely approximate those of dark-matter halos with central cusps, $\rho \sim r^{-1}$, and outer parts, $\rho \sim r^{-4}$. Here we define a non-equilibrium functional, $S_D$, which is maximized for DARKexp and increases monotonically during the evolution towards equilibrium of idealized collisionless systems of the Extended Spherical Infall Model. Systems that undergo more mixing more closely approach DARKexp. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11721-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11721) | **Variability Catalog of Stars Observed During the TESS Prime Mission**  |
|| Tara Fetherolf, et al. -- incl., <mark>Daniel Huber</mark>, <mark>Joseph D. Twicken</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *25 pages, 14 figures, submitted to AAS Journals*|
|**Abstract**| During its 2-year Prime Mission, TESS observed over 232,000 stars at a 2-min cadence across ~70% of the sky. These data provide a record of photometric variability across a range of astrophysically interesting time scales, probing stellar rotation, eclipsing binary systems, and pulsations. We have analyzed the TESS 2-min light curves to identify periodic variability on timescales 0.01-13 days, and explored the results across various stellar properties. We have identified over 40,000 periodic variables with high confidence, and another 50,000 with moderate confidence. These light curves show differences in variability type across the HR diagram, with distinct groupings of rotational, eclipsing, and pulsational variables. We also see interesting patterns across period-luminosity space, with clear correlations between period and luminosity for high-mass pulsators, evolved stars, and contact binary systems, a discontinuity corresponding to the Kraft break, and a lower occurrence of periodic variability in main-sequence stars at a timescale of 1.5 to 2 days. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11738-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11738) | **Abundant Molecular Clouds with CO-dark Envelopes in the Extended  Ultraviolet (XUV) Disk of M83**  |
|| Jin Koda, et al. -- incl., <mark>Amanda M Lee</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *Submitted to AAS Journals in March 2022 (under review)*|
|**Abstract**| We report a CO(3-2) detection of 23 molecular clouds in the extended ultraviolet (XUV) disk of the spiral galaxy M83 with ALMA. The observed 1kpc^2 region is at about 1.24 times the optical radius (R25) of the disk, where CO(2-1) was previously not detected. The detection and non-detection, as well as the level of star formation (SF) activity in the region, can be explained consistently if the clouds have the mass distribution common among Galactic clouds, such as Orion A -- with star-forming dense clumps embedded in thick layers of bulk molecular gas, but in a low-metallicity regime where their outer layers are CO-deficient and CO-dark. The cloud and clump masses, estimated from CO(3-2), range from 8.2x10^2 to 2.3x10^4 Msun and from 2.7x10^2 to 7.5x10^3 Msun, respectively. The most massive clouds appear similar to Orion A in star formation activity as well as in mass, as expected if the cloud mass structure is universal. The overall low SF activity in the XUV disk could be due to the relative shortage of gas in the molecular phase. The clouds are distributed like chains up to 600 pc (or longer) in length, suggesting that the trigger of cloud formation is on large scales. The universal cloud mass structure also justifies the use of high-J CO transitions to trace the total gas mass of clouds, or galaxies, even in the high-z universe. This study is the first demonstration that CO(3-2) is an efficient tracer of molecular clouds even in low-metallicity environments. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11760-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11760) | **The Panchromatic Hubble Andromeda Treasury: Triangulum Extended Region  (PHATTER). IV. Star Cluster Catalog**  |
|| L. Clifton Johnson, et al. -- incl., <mark>Estephani E. TorresVillanueva</mark>, <mark>Benjamin F. Williams</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *24 pages, 15 figures, 9 tables, Accepted to ApJ*|
|**Abstract**| We construct a catalog of star clusters from Hubble Space Telescope images of the inner disk of the Triangulum Galaxy (M33) using image classifications collected by the Local Group Cluster Search, a citizen science project hosted on the Zooniverse platform. We identify 1214 star clusters within the Hubble Space Telescope imaging footprint of the Panchromatic Hubble Andromeda Treasury: Triangulum Extended Region (PHATTER) survey. Comparing this catalog to existing compilations in the literature, 68% of the clusters are newly identified. The final catalog includes multi-band aperture photometry and fits for cluster properties via integrated light SED fitting. The cluster catalog's 50% completeness limit is ~1500 solar masses at an age of 100 Myr, as derived from comprehensive synthetic cluster tests. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11824-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11824) | **Optical IFU Observations of GOALS Sample with KOOLS-IFU on Seimei  Telescope: Initial results of 9 U/LIRGs at $z <$ 0.04**  |
|| Yoshiki Toba, et al. -- incl., <mark>Yoshihiro Ueda</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *12 pages, 8 figures, and 2 tables, accepted for publication in PASJ*|
|**Abstract**| We present ionized gas properties of 9 local ultra/luminous infrared galaxies (U/LIRGs) at $z <$ 0.04 through IFU observations with KOOLS-IFU on Seimei Telescope. The observed targets are drawn from the Great Observatories All-sky LIRG Survey (GOALS), covering a wide range of merger stages. We successfully detect emission lines such as H$\beta$, [OIII]$\lambda$5007, H$\alpha$, [NII]$\lambda\lambda$6549,6583, and [SII]$\lambda\lambda$6717,6731 with a spectral resolution of $R$ = 1500-2000, which provides (i) spatially-resolved ($\sim$200-700 pc) moment map of ionized gas and (ii) diagnostics for active galactic nucleus (AGN) within the central $\sim$3--11 kpc in diameter for our sample. We find that [OIII] outflow that is expected to be driven by AGN tends to be stronger (i) towards the galactic center and (ii) as a sequence of merger stage. In particular, the outflow strength in the late-stage (stage D) mergers is about 1.5 times stronger than that in the early-state (stage B) mergers, which indicates that galaxy mergers could induce AGN-driven outflow and play an important role in the co-evolution of galaxies and supermassive black holes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.11871-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.11871) | **Tight Constraint on the Maximum Mass of Stellar-origin Binary Black  Holes and Evidence for Hierarchical Mergers in Gravitational Wave  Observations**  |
|| Yuan-Zhu Wang, et al. -- incl., <mark>Yi-Zhong Fan</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *16 pages, 8 figures*|
|**Abstract**| The origins of the coalescing binary black holes (BBHs) detected by the advanced LIGO/Virgo are still under debate, and clues may present in the joint mass-spin distribution of these merger events. Here we construct phenomenological models to investigate the BBH population detected in gravitational observations. The data can be well explained by the members originated from two different channels: one is the evolution of field binaries, and the other is the dynamical assembly. We obtain a tight constraint on the maximum mass for events of the stellar-origin, which is $m_{\rm max}=39.4^{+2.6}_{-2.5}M_{\odot}$ at 90\% credibility. This mass cutoff likely arises from the (pulsational) pair-instability supernova explosion and/or stellar winds. We also find that a fraction of $4-17\%$ of dynamical events were hierarchical mergers, and these BHs had an average spin magnitude significantly larger than the first-generation mergers, with ${\rm d}\mu_{\rm a} > 0.4 $ at $99\%$ credibility. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.12089-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.12089) | **WISE/NEOWISE Multi-Epoch Imaging of the Potentially Geminid-related  Asteroids: (3200) Phaethon, 2005 UD and 1999 YC**  |
|| Toshihiro Kasuga, <mark>Joseph R. Masiero</mark> |
|*Appeared on*| *2022-08-26*|
|*Comments*| *Accepted for publication in The Astronomical Journal, 8 tables, 7 figures*|
|**Abstract**| We present space-based thermal infrared observations of the presumably Geminid-associated asteroids: (3200)Phaethon, 2005 UD and 1999 YC using WISE/NEOWISE. The images were taken at the four wavelength bands 3.4$\mu$m(W1),4.6$\mu$m(W2),12$\mu$m(W3),and 22$\mu$m(W4). We find no evidence of lasting mass-loss in the asteroids over the decadal multi-epoch datasets. We set an upper limit to the mass-loss rate in dust of Q<2kg s$^{-1}$ for Phaethon and <0.1kg s$^{-1}$ for both 2005 UD and 1999 YC, respectively, with little dependency over the observed heliocentric distances of R=1.0$-$2.3au. For Phaethon, even if the maximum mass-loss was sustained over the 1000(s)yr dynamical age of the Geminid stream, it is more than two orders of magnitude too small to supply the reported stream mass (1e13$-$14kg). The Phaethon-associated dust trail (Geminid stream) is not detected at R=2.3au, corresponding an upper limit on the optical depth of $\tau$<7e-9. Additionally, no co-moving asteroids with radii r<650m were found. The DESTINY+ dust analyzer would be capable of detecting several of the 10$\mu$m-sized interplanetary dust particles when at far distances(>50,000km) from Phaethon. From 2005 UD, if the mass-loss rate lasted over the 10,000yr dynamical age of the Daytime Sextantid meteoroid stream, the mass of the stream would be ~1e10kg. The 1999 YC images showed neither the related dust trail ($\tau$<2e-8) nor co-moving objects with radii r<170m at R=1.6au. Estimated physical parameters from these limits do not explain the production mechanism of the Geminid meteoroid stream. Lastly, to explore the origin of the Geminids, we discuss the implications for our data in relation to the possibly sodium (Na)-driven perihelion activity of Phaethon. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

20  publications in the last 7 days.
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers